<a href="https://colab.research.google.com/github/yumnaizzati03/KomputasiIntelegensiaTasks/blob/main/TaskWeek7_Yumna_Izzati.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Tugas 7 Komputasi Intelegensia**

Nama: Yumna Izzati

NPM: 2106633191

Model: https://huggingface.co/ayameRushia/roberta-base-indonesian-1.5G-sentiment-analysis-smsa

## **Model Awal**

In [1]:
!pip install transformers

In [2]:
# Use a pipeline as a high-level helper

from transformers import pipeline

pipe = pipeline(
"text-classification",
model="ayameRushia/roberta-base-indonesian-1.5G-sentiment-analysis-smsa"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/990 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/822k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/468k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
pipe("Kuenya sangat lembut dan rasa manisnya juga tidak berlebihan")

[{'label': 'POSITIVE', 'score': 0.9997730851173401}]

In [4]:
pipe("Porsi sayurnya terlalu banyak dibandingkan dagingnya untuk menu sup buntut ini.")

[{'label': 'NEGATIVE', 'score': 0.9997517466545105}]

In [5]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

excel_file_path = '/content/drive/MyDrive/dataset/dataset_sintetis_review_makanan.xlsx'
df = pd.read_excel(excel_file_path)

print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                             Kalimat  Sentimen
0  Makanan di sini sangat lezat, bumbu-bumbunya t...  POSITIVE
1  Sayang sekali, pelayanan lambat dan makanannya...  NEGATIVE
2  Restorannya nyaman, tapi makanannya biasa saja...   NEUTRAL
3  Saya suka suasananya, dan hidangan penutupnya ...  POSITIVE
4  Rasa makanannya kurang sesuai dengan ekspektas...   NEUTRAL


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = df['Kalimat']
y = df['Sentimen']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Get predictions
y_pred = [pipe(text)[0]['label'] for text in X_test]

# 3. Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.6f}")

Accuracy: 0.950000


## **Model dengan Attention Transformers**

In [7]:
# Load model directly

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ayameRushia/roberta-base-indonesian-1.5G-sentiment-analysis-smsa")
model = AutoModelForSequenceClassification.from_pretrained("ayameRushia/roberta-base-indonesian-1.5G-sentiment-analysis-smsa")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
import torch

X = df['Kalimat']
y = df['Sentimen']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

inputs = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()

for i, prediction in enumerate(predicted_labels):
    print(f"Predicted: {prediction}, Actual: {y_test.iloc[i]}")

label_mapping = {0: 'POSITIVE', 1: 'NEUTRAL', 2: 'NEGATIVE'}
predicted_labels = [label_mapping[label] for label in predicted_labels]

accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy:.6f}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Predicted: 2, Actual: NEGATIVE
Predicted: 2, Actual: NEGATIVE
Predicted: 2, Actual: NEGATIVE
Predicted: 0, Actual: POSITIVE
Predicted: 0, Actual: POSITIVE
Predicted: 0, Actual: POSITIVE
Predicted: 0, Actual: POSITIVE
Predicted: 0, Actual: POSITIVE
Predicted: 2, Actual: NEGATIVE
Predicted: 0, Actual: POSITIVE
Predicted: 0, Actual: POSITIVE
Predicted: 0, Actual: POSITIVE
Predicted: 0, Actual: POSITIVE
Predicted: 2, Actual: NEGATIVE
Predicted: 0, Actual: POSITIVE
Predicted: 0, Actual: NEUTRAL
Predicted: 2, Actual: NEGATIVE
Predicted: 0, Actual: POSITIVE
Predicted: 0, Actual: POSITIVE
Predicted: 2, Actual: NEGATIVE
Accuracy: 0.950000


## **Pembahasan**
Didapat hasil bahwa model awal dan model dengan attention transformers memiliki nilai akurasi serupa. Hal ini kemungkinan disebabkan oleh penggunaan dataset AI-generated sehingga tidak memiliki cukup kompleksitas untuk bisa memanfaatkan kinerja model dengan attention transformer secara signifikan. Hal ini membuat kedua model sama-sama dapat mempelajari data dengan baik dan menghasilkan akurasi yang sama.